In [1]:
import pandas as pd
%pylab inline
import json
from sklearn.model_selection import train_test_split 
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [2]:
results = []
with open('customers.json') as json_data:
    for line in json_data:
        results.append(json.loads(line))

In [3]:
# let's see how many customers we have in there
len(results)

168

What we have now is a list of dictionnaries. To easily understand whats coming you need to understand 
two notions. the first is the principle the *get* method for dictionnary objects and the second is simply thatwe can dataframe  a list. Simple as that!<br> Now if you still dont quite understand uncomment and run the examples below

In [4]:
#Eample 1
#books=[{'name':'Harry Potter', 'pages':412, 'year':1997},{'name':'Sun', 'pages':60, 'year':2019},
      #{'name':'Omen', 'pages':341, 'year':2003}]

    
#for i in books:
    #print (i.get('name')) # this will be done in a list comprehension in next cell
    

# Example 2   
#pd.DataFrame(['Dad', 'Mom', 'Gogo'])

In [5]:
print (len(results),'\n')# checking the numbers of customers
print(results[0].keys()) # sniffing the data

168 

dict_keys(['fraudulent', 'customer', 'orders', 'paymentMethods', 'transactions'])


In [6]:
fraud=pd.DataFrame([result.get('fraudulent') for result in results])
fraud.shape

(168, 1)

In [7]:
fraud.head(2)

,0
0,False
1,True


In [8]:
customer=pd.DataFrame([result.get('customer') for result in results])
customer.shape

(168, 5)

In [9]:
customer.tail(3)

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress
165,johnlowery@gmail.com,635.676.0955x524,969zz9zdj8z4gns4sx5p,f82c:811f:8a02:e2d6:79b:fcaa:42de:570b,"548 Bryant Inlet\nVeronicaside, OK 00522"
166,ethompson@jackson-sanders.com,962.300.2250,navessqcnmc9csfjn5cj,203.246.172.244,"16953 Jennifer Springs\nKathybury, IL 02095-3215"
167,dawn05@tucker-brown.com,(089)703-7360x8848,lu4fgcug8dr0dwbcuui1,132.132.95.33,"130 Kimberly Junctions\nEricmouth, KS 59756-2919"


In [10]:
customers=customer.assign(fraudulent=[result.get('fraudulent') for result in results])

In [11]:
customers.head(3)

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,fraudulent
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",False
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",False


In [12]:
transactions = pd.DataFrame([result.get('transactions') for result in results])

In [13]:
transactions.iat[0,0] # sniffing

{'transactionId': 'a9lcj51r',
 'orderId': 'vjbdvd',
 'paymentMethodId': 'wt07xm68b',
 'transactionAmount': 18,
 'transactionFailed': False}

In [14]:
# let's try go
tx=[result.get('transactions') for result in results]
#tx

In [15]:
transac=pd.concat([pd.DataFrame(df).assign(customerId=customer_id) for customer_id, df in enumerate(tx)])

In [16]:
transac.head(3)

,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,customerId
0,a9lcj51r,vjbdvd,wt07xm68b,18.0,False,0
1,y4wcv03i,yp6x27,wt07xm68b,26.0,False,0
0,5mi94sfw,nlghpa,41ug157xz,45.0,False,1


In [17]:
pymts = [result.get('paymentMethods') for result in results]
payment_methods = pd.concat([pd.DataFrame(df).assign(customerid=customer_id) for customer_id, df in enumerate(pymts)]).reset_index(drop=True)

In [18]:
payment_methods.head(3)

,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,paymentMethodIssuer,customerid
0,wt07xm68b,True,card,JCB 16 digit,Citizens First Banks,0
1,y3xp697jx,True,bitcoin,VISA 16 digit,Solace Banks,1
2,6krszxc05,False,card,VISA 16 digit,Vertex Bancorp,1


In [19]:
orders_list = [result.get('orders') for result in results]
orders = pd.concat([pd.DataFrame(df).assign(customerid=customer_id) for customer_id, df in enumerate(orders_list)]).reset_index(drop=True)

In [20]:
orders.head(3)

,orderId,orderAmount,orderState,orderShippingAddress,customerid
0,vjbdvd,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",0
1,yp6x27,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",0
2,nlghpa,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",1


In [21]:
df=(
    transac
    .merge(orders, how='left', on='orderId')
    .merge(customers, how='left', left_on='customerId',
    right_index=True)
)

In [22]:
df.head()

,transactionId,orderId,paymentMethodId,transactionAmount,transactionFailed,customerId,orderAmount,orderState,orderShippingAddress,customerid,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,fraudulent
0,a9lcj51r,vjbdvd,wt07xm68b,18.0,False,0,18.0,pending,"5493 Jones Islands\nBrownside, CA 51896",0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",False
1,y4wcv03i,yp6x27,wt07xm68b,26.0,False,0,26.0,fulfilled,"5493 Jones Islands\nBrownside, CA 51896",0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",False
2,5mi94sfw,nlghpa,41ug157xz,45.0,False,1,45.0,fulfilled,"898 Henry Ports\nNew Keithview, CA 95893-2497",1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True
3,br8ba1nu,uw0eeb,41ug157xz,23.0,False,1,23.0,fulfilled,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True
4,a33145ss,bn44oh,y3xp697jx,43.0,True,1,43.0,fulfilled,"5093 Bryan Forks\nJoshuaton, FM 01565-9801",1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,True


This is sometimes what you have to do as a data scientist. If you are new to json files, please consider consider 
understanding the structure of those files first and try dataframe shorter files before to jump on this notebook. 